In [1]:
import pandas as pd
import numpy as np
try:
    from pgmpy.models import DiscreteBayesianNetwork as BN   # for pgmpy ≥ 1.0.0
except ImportError:
    from pgmpy.models import BayesianModel as BN             # fallback for older versions
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

In [3]:
# Load the dataset (keep heart.csv in the same folder)
df = pd.read_csv("heart.csv").replace('?', np.nan).dropna()

# Rename target column if needed
for t in ['target', 'num']:
    if t in df.columns and 'heartdisease' not in df.columns:
        df = df.rename(columns={t: 'heartdisease'})

# Keep only important columns
cols = ['age','sex','trestbps','fbs','exang','restecg','thalach','chol','heartdisease']
df = df[[c for c in cols if c in df.columns]].copy()

df.head()

,age,sex,trestbps,fbs,exang,restecg,thalach,chol,heartdisease
0,63,1,145,1,0,0,150,233,1
1,37,1,130,0,0,1,187,250,1
2,41,0,130,0,0,0,172,204,1
3,56,1,120,0,0,1,178,236,1
4,57,0,120,0,1,1,163,354,1


In [4]:
if 'age' in df:       df['age'] = (df['age'].astype(float) > 50).astype(int)
if 'trestbps' in df:  df['trestbps'] = (df['trestbps'].astype(float) > 130).astype(int)
if 'thalach' in df:   df['thalach'] = (df['thalach'].astype(float) < 150).astype(int)  # 1 = low
if 'chol' in df:      df['chol'] = (df['chol'].astype(float) > 200).astype(int)
df = df.dropna().astype(int)

df.head()

,age,sex,trestbps,fbs,exang,restecg,thalach,chol,heartdisease
0,1,1,1,1,0,0,0,1,1
1,0,1,0,0,0,1,0,1,1
2,0,0,0,0,0,0,0,1,1
3,1,1,0,0,0,1,0,1,1
4,1,0,0,0,1,1,0,1,1


In [5]:
features = [c for c in df.columns if c != 'heartdisease']
model = BN([('heartdisease', f) for f in features])

# Learn CPDs
model.fit(df, estimator=MaximumLikelihoodEstimator)
infer = VariableElimination(model)

print("Learning CPDs using Maximum Likelihood Estimators...\n")
print("Inferencing with Bayesian Network:")

C:\Users\tanma\anaconda3\Lib\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


Learning CPDs using Maximum Likelihood Estimators...

Inferencing with Bayesian Network:


In [6]:
def bin_evidence(evidence):
    b = {}
    for k, v in evidence.items():
        if k == 'age': b[k] = int(float(v) > 50)
        elif k == 'trestbps': b[k] = int(float(v) > 130)
        elif k == 'chol': b[k] = int(float(v) > 200)
        elif k == 'thalach': b[k] = int(float(v) < 150)
        else: b[k] = int(v)
    return b

In [7]:
print("\n1. Probability of HeartDisease given Age = 28")
print(infer.query(['heartdisease'], evidence=bin_evidence({'age': 28})))

print("\n2. Probability of HeartDisease given Cholesterol = 100")
print(infer.query(['heartdisease'], evidence=bin_evidence({'chol': 100})))


1. Probability of HeartDisease given Age = 28
+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              0.3053 |
+-----------------+---------------------+
| heartdisease(1) |              0.6947 |
+-----------------+---------------------+

2. Probability of HeartDisease given Cholesterol = 100
+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              0.4118 |
+-----------------+---------------------+
| heartdisease(1) |              0.5882 |
+-----------------+---------------------+


C:\Users\tanma\anaconda3\Lib\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(
